# 1. Introdução
Esta seção tem como objetivo preparar e organizar o ambiente em que o MVP será construído, isto é, definir as bibliotecas necessárias, eventuais funções que precisem ser construídas e estruturar o ambiente onde os dados serão armazenados.

## 1.1 Bibliotecas

In [0]:
# Bibliotecas utilizadas ao longo do trabalho

import pandas as pd
import glob
import os

from pyspark.sql.functions import lit #sql functions

## 1.2 Funções

In [0]:
# Funções utilizadas ao longo do trabalho




Significado das funções em pyspark:

- LIT: permite adicionar valores constantes a um dataframe. É útil quando precisamos adicionar uma coluna fixa a um dataframe.

## 1.3 Preparação do ambiente
Neste MVP, pretendo simular um ambiente com arquitetura em camadas, sendo elas:

- Stage: armazenamento dos dados brutos (camada obrigatória)
- Intermediate: armazenamento de dados parcialmente tratados e eventualmente relacionados (camada opcional)
- Data Warehouse: armazenamento de dados em modelagem Star Schema (camada obrigatória)

In [0]:
%sql
-- Criação das camadas da arquitetura

create database if not exists stage;          -- ambiente para armazenar os dados brutos
create database if not exists intermediate;   -- ambiente para armazenar os dados parcialmente tratados
create database if not exists datawarehouse;  -- ambiente para armazenar os dados modelados entre fatos e dimensões

show databases;

databaseName
datawarehouse
default
intermediate
stage


In [0]:
%sql
-- Exibição das tabelas nas camadas criadas

-- show tables in stage;
-- show tables in intermediate;
show tables in datawarehouse;


database,tableName,isTemporary
,vw_gastronomades_data_history,true
,vw_youtube_202211,true
,vw_youtube_202212,true
,vw_youtube_202301,true
,vw_youtube_202302,true
,vw_youtube_202303,true
,vw_youtube_202304,true
,vw_youtube_202305,true
,vw_youtube_202306,true
,vw_youtube_202307,true


# 2. Contextualização do problema
Nesta seção, faremos a contextualização do problema que desejamos resolver com este MVP, bem como seu objetivo e eventuais perguntas de interesse que gostaríamos de responder.


## 2.1 Objetivo

Este MVP tem como objetivo realizar uma análise histórica dos dados do **Gastronomades**, um dos meus canais do YouTube, a fim de compreender o desempenho do canal através de métricas disponibilizadas pelo próprio **YouTube Analytics**, que é a ferramenta oficial da plataforma para monitoramento de dados.

O propósito é que consigamos obter insights sobre o crescimento do canal, o impacto dos vídeos e a efetividade das estratégias de conteúdo aplicadas em **outubro de 2024**.

Em setembro de 2024, tomamos a decisão de **excluir um vídeo** que acreditávamos estar distorcendo as métricas por se tratar de um conteúdo muito diferente do que normalmente criamos para o canal. Além disso, **aprimoramos a qualidade dos vídeos**, tanto na roteirização quanto no estilo de edição, incluindo melhorias na estrutura das _thumbnails_, transições mais refinadas, padronização de _placeholders_, escolha consistente de fontes e letras, além da criação de uma paleta de cores específica para fortalecer a identidade visual do canal.

Portanto, este MVP também busca avaliar se essas mudanças **impactaram positivamente o nosso canal**, analisando se os vídeos publicados após esse período apresentaram melhores resultados em comparação aos anteriores.

## 2.2 Perguntas de interesse

A análise dos dados do **Gastronomades** será guiada pelas seguintes perguntas:

#### 🎥 Vídeos  
- Quantos vídeos o canal possui atualmente?  
- Quantos vídeos do tipo "short" o canal possui atalmente?
- Quantos vídeos de longa duração o canal possui atalmente?
- Quantos vídeos foram postados por período (exceto hoje)?  
- Qual o tempo médio de visualização por vídeo atualmente?  
- Qual o tempo médio de visualização por período?  
- Quais vídeos têm o maior tempo de retenção?  

#### 📋 Playlists  
- Quantas playlists o canal possui atualmente?  
- Quais playlists possuem mais vídeos?  
- Existe alguma relação entre a organização dos vídeos em playlists e o número de visualizações?  

#### 👥 Inscritos  
- Qual o número de inscritos do canal por período?  
- Qual vídeo gerou o maior número de inscritos até o momento?  
- Houve aumento na taxa de inscritos após a exclusão do vídeo e a implementação das novas estratégias?  

### 👍 Curtidas  
- Qual o número médio de likes por vídeo?  
- Qual a taxa média de likes por vídeo?  
- Qual vídeo possui a maior taxa média de likes?  
- A melhoria na qualidade e identidade visual dos vídeos impactou no número de curtidas?  

#### 👀 Impressões e Cliques  
- Qual o número de impressões por período?  
- Qual o número médio de impressões por vídeo?  
- Quais vídeos têm tido maior número de impressões?  
- Qual a taxa de cliques (_CTR - Click Through Rate_) por período?  
- Qual é a taxa média de cliques por vídeo?  
- Quais vídeos têm tido a maior taxa de cliques?  
- A mudança no design das _thumbnails_ impactou a taxa de cliques?  

#### 💰 Receita  
- Qual a receita do canal por período?  
- Qual vídeo gerou a maior fonte de receita no período?  
- Qual a receita média por vídeo do canal?  
- Qual é a RPM (Receita por cada 1000 visualizações) média do canal?  
- Quais vídeos têm tido maior RPM?  
- Houve aumento na receita após a exclusão do vídeo e a adoção das novas estratégias?  

#### ⏳ Tempo de visualização  
- Qual o tempo médio de visualização por vídeo atualmente?  
- Qual o tempo médio de visualização por período?  
- Quais vídeos têm tido maior tempo de visualização atualmente?  
- Houve impacto na retenção de público com as novas estratégias?  

### 📆 Períodos de análise  
Os dados serão analisados considerando diferentes recortes temporais para entender melhor a evolução do canal:  
- Hoje  
- Mês anterior 
- Últimos 3 meses  
- Todo o período de dados disponível  


## 2.3 Obtenção dos Dados  

Para este MVP, os dados foram extraídos manualmente do **YouTube Analytics** em formato **CSV**. Esse método foi escolhido devido à sua praticidade e rapidez na obtenção das informações necessárias para a análise.  

Entretanto, o objetivo futuro é evoluir este projeto para que os dados sejam consumidos diretamente da **API do YouTube**, permitindo assim uma integração automatizada e em tempo real, o que possibilitará análises mais dinâmicas e atualizadas de forma contínua.  

Foram extraídos dados desde a criação do canal, isto é, entre novembro de 2022 e março de 2025, no entanto, para esta primeira versão consideraremos apenas o período dos últimos 12 meses fechados.

# 3. ETL dos dados
Esta seção tem como objetivo preparar e organizar o ambiente em que o MVP será construído, isto é, definir as bibliotecas necessárias, eventuais funções que precisem ser construídas e estruturar o ambiente onde os dados serão armazenados.

### 3.1 DBFS
Aqui, faremos a validação dos diretórios do DataBricks File System (DBFS) a fim de inspecionar eventuais duplicidades de arquivos.

In [0]:
# DataBricks File System

# Lista conteúdo de diretórios
#dbutils.fs.ls('/')                                                 #lista o conteúdo do diretório raiz do DBFS
#dbutils.fs.ls('/FileStore')                                         #lista o conteúdo do diretório FileStore

# Exclui um diretório e seu conteúdo (recursivamente)
#dbutils.fs.rm('dbfs:/FileStore/shared_uploads/', recurse=True)    #deleta os arquivos do diretório em questão
dbutils.fs.rm('/FileStore/puc_rio/mvp_engenharia_de_dados/202504_kpis.csv', recurse=False)    #excluir um arquivo específico
dbutils.fs.rm('/FileStore/puc_rio/mvp_engenharia_de_dados/202504_kpis-1.csv', recurse=False)    #excluir um arquivo específico
dbutils.fs.rm('/FileStore/puc_rio/mvp_engenharia_de_dados/202504_kpis-2.csv', recurse=False)    #excluir um arquivo específico
dbutils.fs.rm('/FileStore/puc_rio/mvp_engenharia_de_dados/202503_kpis.csv', recurse=False)    #excluir um arquivo específico



Out[1]: True

In [0]:
# DataBricks File System

# Remove todos os arquivos do diretório

files = dbutils.fs.ls("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/")

for file in files:
    if file.name.endswith(".csv"):
        dbutils.fs.rm(file.path)


In [0]:
#checa 

#dbutils.fs.ls('/FileStore')  
dbutils.fs.ls('/FileStore/puc_rio') 

Out[3]: [FileInfo(path='dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/', name='mvp_engenharia_de_dados/', size=0, modificationTime=0)]

### 3.2 Input
Como mencionado anteriormente, os dados foram extraídos manualmente do YouTube Analytics em formato CSV. Dessa forma, após realizarmos o armazenamento dos arquivos mensais no DBFS, faremos a leitura de cada um deles para posterior armazenamento em dataframes via Spark. Uma vez que os dados tenham sido materializados, criaremos views temporárias para cada um dos arquivos a fim de conseguirmos manipulá-los em SQL. 


In [0]:
# Armazena o conteúdo dos arquivos csv em dataframes via Spark

#2022
youtube_202211 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202211_kpis.csv")
youtube_202212 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202212_kpis.csv")

#2023
youtube_202301 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202301_kpis.csv")
youtube_202302 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202302_kpis.csv")
youtube_202303 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202303_kpis.csv")
youtube_202304 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202304_kpis.csv")
youtube_202305 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202305_kpis.csv")
youtube_202306 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202306_kpis.csv")
youtube_202307 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202307_kpis.csv")
youtube_202308 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202308_kpis.csv")
youtube_202309 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202309_kpis.csv")
youtube_202310 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202310_kpis.csv")
youtube_202311 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202311_kpis.csv")
youtube_202312 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202312_kpis.csv")

#2024
youtube_202401 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202401_kpis.csv")
youtube_202402 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202402_kpis.csv")
youtube_202403 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202403_kpis.csv")
youtube_202404 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202404_kpis.csv")
youtube_202405 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202405_kpis.csv")
youtube_202406 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202406_kpis.csv")
youtube_202407 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202407_kpis.csv")
youtube_202408 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202408_kpis.csv")
youtube_202409 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202409_kpis.csv")
youtube_202410 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202410_kpis.csv")
youtube_202411 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202411_kpis.csv")
youtube_202412 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202412_kpis.csv")

#2025
youtube_202501 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202501_kpis.csv")
youtube_202502 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202502_kpis.csv")
youtube_202503 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202503_kpis.csv")
youtube_202504 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/puc_rio/mvp_engenharia_de_dados/202504_kpis.csv")

#Hoje


In [0]:
# Prepara os dados para serem tratados em SQL
# No DataBricks, para rodar queries em SQL é necessário que os dados sejam tabelas registradas no metastore OU uma view temporária

# Constrói views temporárias para posteriormente armazená-las na camada stage

#2022
youtube_202211.createOrReplaceTempView("vw_youtube_202211")
youtube_202212.createOrReplaceTempView("vw_youtube_202212")

#2023
youtube_202301.createOrReplaceTempView("vw_youtube_202301")
youtube_202302.createOrReplaceTempView("vw_youtube_202302")
youtube_202303.createOrReplaceTempView("vw_youtube_202303")
youtube_202304.createOrReplaceTempView("vw_youtube_202304")
youtube_202305.createOrReplaceTempView("vw_youtube_202305")
youtube_202306.createOrReplaceTempView("vw_youtube_202306")
youtube_202307.createOrReplaceTempView("vw_youtube_202307")
youtube_202308.createOrReplaceTempView("vw_youtube_202308")
youtube_202309.createOrReplaceTempView("vw_youtube_202309")
youtube_202310.createOrReplaceTempView("vw_youtube_202310")
youtube_202311.createOrReplaceTempView("vw_youtube_202311")
youtube_202312.createOrReplaceTempView("vw_youtube_202312")

#2024
youtube_202401.createOrReplaceTempView("vw_youtube_202401")
youtube_202402.createOrReplaceTempView("vw_youtube_202402")
youtube_202403.createOrReplaceTempView("vw_youtube_202403")
youtube_202404.createOrReplaceTempView("vw_youtube_202404")
youtube_202405.createOrReplaceTempView("vw_youtube_202405")
youtube_202406.createOrReplaceTempView("vw_youtube_202406")
youtube_202407.createOrReplaceTempView("vw_youtube_202407")
youtube_202408.createOrReplaceTempView("vw_youtube_202408")
youtube_202409.createOrReplaceTempView("vw_youtube_202409")
youtube_202410.createOrReplaceTempView("vw_youtube_202410")
youtube_202411.createOrReplaceTempView("vw_youtube_202411")
youtube_202412.createOrReplaceTempView("vw_youtube_202412")

#2025
youtube_202501.createOrReplaceTempView("vw_youtube_202501")
youtube_202502.createOrReplaceTempView("vw_youtube_202502")
youtube_202503.createOrReplaceTempView("vw_youtube_202503")
youtube_202504.createOrReplaceTempView("vw_youtube_202504")

### 3.3 Manipulação e transformação
Para que possamos simplificar o processo de manipulação e modelagem dos dados brutos, faremos a consolidação dos dados mensais em uma única view temporária. Além disso, também faremos algumas tratativas, como padronização dos nomes dos atributos e eventuais limpezas de dados.

In [0]:
%sql
-- ETL
-- Inclui a coluna de mês de referência nas views e as unifica os dados mensais em uma única view temporária

create or replace temporary view vw_gastronomades_data_history as (

-- 202211
select 
  '2022-11-01' as reference_date
, yt202211.* 
from vw_youtube_202211 as yt202211

union all 

-- 202212
select 
  '2022-12-01' as reference_date
, yt202212.* 
from vw_youtube_202212 as yt202212  

union all  

-- 202301
select 
  '2023-01-01' as reference_date
, yt202301.* 
from vw_youtube_202301 as yt202301

union all

-- 202302
select 
  '2023-02-01' as reference_date
, yt202302.* 
from vw_youtube_202302 as yt202302

union all

-- 202303
select 
  '2023-03-01' as reference_date
, yt202303.* 
from vw_youtube_202303 as yt202303

union all

-- 202304
select 
  '2023-04-01' as reference_date
, yt202304.* 
from vw_youtube_202304 as yt202304

union all

-- 202305
select 
  '2023-05-01' as reference_date
, yt202305.* 
from vw_youtube_202305 as yt202305

union all 

-- 202306
select 
  '2023-06-01' as reference_date
, yt202306.* 
from vw_youtube_202306 as yt202306

union all 

-- 202307
select 
  '2023-07-01' as reference_date
, yt202307.* 
from vw_youtube_202307 as yt202307

union all 

-- 202308
select 
  '2023-08-01' as reference_date
, yt202308.* 
from vw_youtube_202308 as yt202308

union all 

-- 202309
select 
  '2023-09-01' as reference_date
, yt202309.* 
from vw_youtube_202309 as yt202309

union all 

-- 202310
select 
  '2023-10-01' as reference_date
, yt202310.* 
from vw_youtube_202310 as yt202310

union all 

-- 202311
select 
  '2023-11-01' as reference_date
, yt202311.* 
from vw_youtube_202311 as yt202311

union all 

-- 202312
select 
  '2023-12-01' as reference_date
, yt202312.* 
from vw_youtube_202312 as yt202312  

union all

-- 202401
select 
  '2024-01-01' as reference_date
, yt202401.* 
from vw_youtube_202401 as yt202401

union all

-- 202402
select 
  '2024-02-01' as reference_date
, yt202402.* 
from vw_youtube_202402 as yt202402

union all

-- 202403
select 
  '2024-03-01' as reference_date
, yt202403.* 
from vw_youtube_202403 as yt202403

union all

-- 202404
select 
  '2024-04-01' as reference_date
, yt202404.* 
from vw_youtube_202404 as yt202404

union all

-- 202405
select 
  '2024-05-01' as reference_date
, yt202405.* 
from vw_youtube_202405 as yt202405

union all 

-- 202406
select 
  '2024-06-01' as reference_date
, yt202406.* 
from vw_youtube_202406 as yt202406

union all 

-- 202407
select 
  '2024-07-01' as reference_date
, yt202407.* 
from vw_youtube_202407 as yt202407

union all 

-- 202408
select 
  '2024-08-01' as reference_date
, yt202408.* 
from vw_youtube_202408 as yt202408

union all 

-- 202409
select 
  '2024-09-01' as reference_date
, yt202409.* 
from vw_youtube_202409 as yt202409

union all 

-- 202410
select 
  '2024-10-01' as reference_date
, yt202410.* 
from vw_youtube_202410 as yt202410

union all 

-- 202411
select 
  '2024-11-01' as reference_date
, yt202411.* 
from vw_youtube_202411 as yt202411

union all 

-- 202412
select 
  '2024-12-01' as reference_date
, yt202412.* 
from vw_youtube_202412 as yt202412

union all 

-- 202501
select 
  '2025-01-01' as reference_date
, yt202501.* 
from vw_youtube_202501 as yt202501

union all

-- 202502
select 
  '2025-02-01' as reference_date
, yt202502.* 
from vw_youtube_202502 as yt202502

union all

-- 202503
select 
  '2025-03-01' as reference_date
, yt202503.* 
from vw_youtube_202503 as yt202503


union all

-- 202504
select 
  '2025-04-01' as reference_date
, yt202504.* 
from vw_youtube_202504 as yt202504


);

-- Leitura da view unificada
select
*
from vw_gastronomades_data_history
limit 3;

reference_date,Conteúdo,Título do vídeo,Horário de publicação do vídeo,Duração,Estimativa de reproduções monetizadas,Cliques por elemento da tela final mostrado (%),Cliques no elemento da tela final,Elementos da tela final exibidos,Cliques por card mostrado (%),Cards mostrados,Comentários adicionados,Inscrições perdidas,Compartilhamentos,"""""""Gostei"""" (vs. """"Não gostei"""") (%)""",CPM (USD),RPM (USD),Porcentagem visualizada média (%),Duração média da visualização,Visualizações,Tempo de exibição (horas),Inscritos,Receita estimada (USD),Impressões,Taxa de cliques de impressões (%)
2022-11-01,Total,null,null,null,0,0.77,2,260,0,0,5,6,20,98.77,0,0,20.36,0:04:52,3187,259.1579,41,0,49050,4.4
2022-11-01,K9vEMqU5ryE,LAVANDÁRIO PEDRA AZUL - ES,"Nov 15, 2022",497,null,12.5,1,8,null,0,3,0,0,100,null,null,21.05,0:01:44,55,1.5986,2,null,588,5.1
2022-11-01,xKyysw_ic0c,ARRAIAL DO CABO - ROTEIRO COMPLETO DE 02 DIAS,"Nov 5, 2022",1455,null,0.4,1,252,null,0,2,3,20,98.63,null,null,20.32,0:04:55,3131,257.1505,31,null,48460,4.39


In [0]:
%sql
-- ETL
-- Padroniza a nomenclatura das colnas da base agregada e carrega o resultado da query como uma tabela na camada stage


-- Exclui a tabela, caso exista
drop table if exists stage.stg__gastronomades_data_history;


-- Cria a tabela de dados brutos na camada stage
create or replace table stage.stg__gastronomades_data_history as (

select
  gdh.reference_date                                      as reference_date
--
, gdh.`Conteúdo`                                          as content_id                         -- content
, gdh.`Título do vídeo`                                   as content_title                      -- content
, gdh.`Horário de publicação do vídeo`                    as content_publish_date               -- content
, gdh.`Duração`                                           as content_duration                   -- content
--
, gdh.`Receita estimada (USD)`                            as estimated_revenue                  -- revenue
, gdh.`Estimativa de reproduções monetizadas`             as estimated_monetized_playbacks      -- revenue
, gdh.`CPM (USD)`                                         as cost_per_thousand_impressions      -- revenue
, gdh.`RPM (USD)`                                         as revenue_per_thousand_views         -- revenue
--
, gdh.`Impressões`                                        as impressions                        -- view
, gdh.`Visualizações`                                     as views                              -- view
, gdh.`Taxa de cliques de impressões (%)`                 as click_through_rate                 -- view 
, gdh.`Duração média da visualização`                     as avg_duration_view_rate             -- view
, gdh.`Tempo de exibição (horas)`                         as watch_time                         -- view
, gdh.`Porcentagem visualizada média (%)`                 as avg_view_rate                      -- view
--
, gdh.`Inscritos`                                         as gained_subscribers                 -- engagement
, gdh.`Inscrições perdidas`                               as lost_subscribers                   -- engagement
, gdh.`Compartilhamentos`                                 as shares                             -- engagement
, gdh.`Comentários adicionados`                           as added_comments                     -- engagement
, gdh.`"""Gostei"" (vs. ""Não gostei"") (%)"`             as like_rate                          -- engagement
, gdh.`Cards mostrados`                                   as shown_cards
--
, gdh.`Cliques por card mostrado (%)`                     as card_click_rate                    -- engagement
, gdh.`Elementos da tela final exibidos`                  as end_screen_elements_shown          -- engagement
, gdh.`Cliques no elemento da tela final`                 as end_screen_clicks                  -- engagement
, gdh.`Cliques por elemento da tela final mostrado (%)`   as end_screen_click_rate              -- engagement
from vw_gastronomades_data_history as gdh
where 1=1
and gdh.`Conteúdo` <> 'Total' -- remove linhas totalizadoras

);

-- Visualiza os dados na tabela criada
select 
*
from stage.stg__gastronomades_data_history
order by 
  content_publish_date desc 
limit 5;

reference_date,content_id,content_title,content_publish_date,content_duration,estimated_revenue,estimated_monetized_playbacks,cost_per_thousand_impressions,revenue_per_thousand_views,impressions,views,click_through_rate,avg_duration_view_rate,watch_time,avg_view_rate,gained_subscribers,lost_subscribers,shares,added_comments,like_rate,shown_cards,card_click_rate,end_screen_elements_shown,end_screen_clicks,end_screen_click_rate
2023-11-01,H5km-VuXwYc,Onde se HOSPEDAR na ORLA BARDOT | Roteiros de Charme em Búzios,"Sep 5, 2023",675,0.287,64,4.509,2.929,477,98,16.77,0:03:42,6.068,33.02,3,0,1,1,100,0,null,25,1,4
2024-05-01,H5km-VuXwYc,Onde se HOSPEDAR na ORLA BARDOT | Roteiros de Charme em Búzios,"Sep 5, 2023",675,0.147,53,3.036,2.1,481,70,10.81,0:03:26,4.0081,30.54,0,0,2,0,80,0,null,27,0,0
2024-07-01,H5km-VuXwYc,Onde se HOSPEDAR na ORLA BARDOT | Roteiros de Charme em Búzios,"Sep 5, 2023",675,0.144,47,3.165,2.149,430,67,9.3,0:03:25,3.8277,30.47,0,0,1,0,100,0,null,18,0,0
2024-11-01,H5km-VuXwYc,Onde se HOSPEDAR na ORLA BARDOT | Roteiros de Charme em Búzios,"Sep 5, 2023",675,0.193,86,2.031,1.664,533,116,15.57,0:03:55,7.5756,34.83,1,0,3,0,100,0,null,44,2,4.55
2025-03-01,H5km-VuXwYc,Onde se HOSPEDAR na ORLA BARDOT | Roteiros de Charme em Búzios,"Sep 5, 2023",675,0.225,48,3.779,2.744,483,82,10.14,0:03:00,4.1068,26.71,0,0,1,0,100,0,null,18,0,0


### 3.4 Inspeção dos dados
Nesta etapa, faremos a inspeção de cada um dos atributos. A ideia é que consigamos descobrir e entender as características individuais de cada um deles, bem como formato, range, integridade, constraints, entre outras. Algumas informações obtidas nesta etapa serão utilizadas para enriquecer o catálogo de dados na próxima seção. 


In [0]:
%sql

-- Verifica os tipos de dados na origem

describe stage.stg__gastronomades_data_history;

col_name,data_type,comment
reference_date,string,null
content_id,string,null
content_title,string,null
content_publish_date,string,null
content_duration,string,null
estimated_revenue,string,null
estimated_monetized_playbacks,string,null
cost_per_thousand_impressions,string,null
revenue_per_thousand_views,string,null
impressions,string,null


In [0]:
%sql

-- ETL
-- Inspeção geral dos registros

-- Inspeção 1: Contagem total
select 
  count(*)                            as row_count       -- count de registros
, count(distinct gdh.content_id)      as content_count   -- count de conteúdos
, count(distinct gdh.reference_date)  as period_count    -- count de periodos
from stage.stg__gastronomades_data_history as gdh;


-- Inspeção 2: Contagem por período
select 
  gdh.reference_date                  as period
, count(*)                            as row_count       -- count de registros
, count(distinct gdh.content_id)      as content_count   -- count de conteúdos
from stage.stg__gastronomades_data_history as gdh
group by 
  gdh.reference_date
order by
  gdh.reference_date;

In [0]:
%sql

-- Validação do atributo `Conteúdo` (content_id) e `Título do vídeo` (content_title)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and (gdh.content_id is null or gdh.content_title is null);

In [0]:
%sql

-- Validação do atributo `Conteúdo` (content_id) e `Título do vídeo` (content_title)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and (gdh.content_id is null or gdh.content_title is null);


-- Contagem de duplicatas (se resultado = vazio, então nenhuma duplicidade)
select
  gdh.reference_date
, gdh.content_id  
, count(gdh.content_id)
from stage.stg__gastronomades_data_history as gdh
group by 
  gdh.reference_date
, gdh.content_id  
having count(gdh.content_id) > 1;


-- Contagem de títulos por conteúdo (se resultado = vazio, então nenhum conteúdo com mais de um título)
select
  gdh.content_id 
, count(distinct gdh.content_title) as title_count   
from stage.stg__gastronomades_data_history as gdh
group by 
  gdh.content_id 
having 
  count(distinct gdh.content_title) > 1; 

In [0]:
%sql

-- Validação do atributo `Horário de publicação do vídeo` (content_publish_date)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.content_publish_date is null;


-- Mínimo e Máximo (atributo tipo data)
select
  min(to_date(from_unixtime(unix_timestamp(gdh.content_publish_date, 'MMM d, yyyy'))))  as content_publish_date_min
, max(to_date(from_unixtime(unix_timestamp(gdh.content_publish_date, 'MMM d, yyyy'))))  as content_publish_date_max
from stage.stg__gastronomades_data_history as gdh


In [0]:
%sql

-- Validação do atributo `Duração` (content_duration)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.content_duration is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.content_duration as int)) as content_duration_min
, max(cast(gdh.content_duration as int)) as content_duration_max
from stage.stg__gastronomades_data_history as gdh


In [0]:
%sql

-- Validação do atributo `Receita estimada (BRL)` (estimated_revenue)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.estimated_revenue is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.estimated_revenue as float)) as estimated_revenue_min
, max(cast(gdh.estimated_revenue as float)) as estimated_revenue_max
from stage.stg__gastronomades_data_history as gdh;


In [0]:
%sql

-- Validação do atributo `Estimativa de reproduções monetizadas` (estimated_monetized_playbacks)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.estimated_monetized_playbacks is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.estimated_monetized_playbacks as int)) as estimated_monetized_playbacks_min
, max(cast(gdh.estimated_monetized_playbacks as int)) as eestimated_monetized_playbacks_max
from stage.stg__gastronomades_data_history as gdh;


In [0]:
%sql

-- Validação do atributo `CPM (BRL)` (cost_per_thousand_impressions)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.cost_per_thousand_impressions is null;



-- Mínimo e Máximo (atributo tipo float)
select
  min(cast(gdh.cost_per_thousand_impressions as float)) as cost_per_thousand_impressions_min
, max(cast(gdh.cost_per_thousand_impressions as float)) as cost_per_thousand_impressions_max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.cost_per_thousand_impressions as float) = (
  
  select max(cast(gdh.cost_per_thousand_impressions as float)) from stage.stg__gastronomades_data_history as gdh
  
);

In [0]:
%sql

-- Validação do atributo `RPM (BRL)` (revenue_per_thousand_impressions)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.revenue_per_thousand_views is null;



-- Mínimo e Máximo (atributo tipo float)
select
  min(cast(gdh.revenue_per_thousand_views as float)) as revenue_per_thousand_views_min
, max(cast(gdh.revenue_per_thousand_views as float)) as revenue_per_thousand_views_max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.revenue_per_thousand_views as float) = (
  
  select max(cast(gdh.revenue_per_thousand_views as float)) from stage.stg__gastronomades_data_history as gdh
  
);

In [0]:
%sql

-- Validação do atributo `Impressões` (impressions)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.impressions is null;


-- Mínimo e Máximo (atributo tipo float)
select
  min(cast(gdh.impressions as int)) as min
, max(cast(gdh.impressions as int)) as max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.impressions as int) = (
  
  select max(cast(gdh.impressions as int)) from stage.stg__gastronomades_data_history as gdh
  
);


In [0]:
%sql

-- Validação do atributo `Visualizações` (views)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.views is null;



-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.views as int)) as min
, max(cast(gdh.views as int)) as max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.views as int) = (
  
  select max(cast(gdh.views as int)) from stage.stg__gastronomades_data_history as gdh
  
);



In [0]:
%sql

-- Validação do atributo `Taxa de cliques de impressões (%)` (click_through_rate)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.click_through_rate is null;



-- Mínimo e Máximo (atributo tipo float)
select
  min(cast(gdh.click_through_rate as float)) as min
, max(cast(gdh.click_through_rate as float)) as max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.click_through_rate as float) = (
  
  select max(cast(gdh.click_through_rate as float)) from stage.stg__gastronomades_data_history as gdh
  
);


In [0]:
%sql

-- Validação do atributo `Duração média da visualização)` (avg_duration_view_rate)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.avg_duration_view_rate is null;



-- Mínimo e Máximo (atributo tipo string)
select
  min(gdh.avg_duration_view_rate) as min
, max(avg_duration_view_rate) as max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.avg_duration_view_rate = (
  
  select max(gdh.avg_duration_view_rate) from stage.stg__gastronomades_data_history as gdh
  
);




In [0]:
%sql

-- Validação do atributo `Tempo de exibição (horas)` (watch_time)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.watch_time is null;



-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.watch_time as float)) as min
, max(cast(gdh.watch_time as float)) as max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.watch_time as float) = (
  
  select max(cast(gdh.watch_time as float)) from stage.stg__gastronomades_data_history as gdh
  
);

In [0]:
%sql

-- Validação do atributo `Porcentagem visualizada média (%)` (avg_view_rate)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.avg_view_rate is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.avg_view_rate as float)) as min
, max(cast(gdh.avg_view_rate as float)) as max
from stage.stg__gastronomades_data_history as gdh;




-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.avg_view_rate as float) = (
  
  select max(cast(gdh.avg_view_rate as float)) from stage.stg__gastronomades_data_history as gdh
  
);



In [0]:
%sql

-- Validação do atributo `Inscritos` (gained_subscribers)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.gained_subscribers is null;



-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.gained_subscribers as int)) as min
, max(cast(gdh.gained_subscribers as int)) as max
from stage.stg__gastronomades_data_history as gdh;


-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.gained_subscribers as int) = (
  
  select max(cast(gdh.gained_subscribers as int)) from stage.stg__gastronomades_data_history as gdh
  
);

In [0]:
%sql

-- Validação do atributo `Inscrições perdidas` (lost_subscribers)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.lost_subscribers is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.lost_subscribers as int)) as min
, max(cast(gdh.lost_subscribers as int)) as max
from stage.stg__gastronomades_data_history as gdh;


-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.lost_subscribers as int) = (
  
  select max(cast(gdh.lost_subscribers as int)) from stage.stg__gastronomades_data_history as gdh
  
);

In [0]:
%sql

-- Validação do atributo `Compartilhamentos` (shares)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.shares is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.shares as int)) as min
, max(cast(gdh.shares as int)) as max
from stage.stg__gastronomades_data_history as gdh;


-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.shares as int) = (
  
  select max(cast(gdh.shares as int)) from stage.stg__gastronomades_data_history as gdh
  
);


In [0]:
%sql

-- Validação do atributo `Comentários adicionados` (added_comments)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.added_comments is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.added_comments as int)) as min
, max(cast(gdh.added_comments as int)) as max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.added_comments as int) = (
  
  select max(cast(gdh.added_comments as int)) from stage.stg__gastronomades_data_history as gdh
  
);



In [0]:
%sql
-- Validação do atributo `"""Gostei"" (vs. ""Não gostei"") (%)"` (like_rate)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.like_rate is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.like_rate as float)) as min
, max(cast(gdh.like_rate as float)) as max
from stage.stg__gastronomades_data_history as gdh;


-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.like_rate as float) = (
  
  select max(cast(gdh.like_rate as float)) from stage.stg__gastronomades_data_history as gdh
  
);

In [0]:
%sql

-- Validação do atributo `Cards mostrados` (shown_cards)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.shown_cards is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.shown_cards as int)) as min
, max(cast(gdh.shown_cards as int)) as max
from stage.stg__gastronomades_data_history as gdh;


-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.shown_cards as int) = (
  
  select max(cast(gdh.shown_cards as int)) from stage.stg__gastronomades_data_history as gdh
  
);

In [0]:
%sql
-- Validação do atributo `Cliques por card mostrado (%)` (card_click_rate)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.card_click_rate is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.card_click_rate as float)) as min
, max(cast(gdh.card_click_rate as float)) as max
from stage.stg__gastronomades_data_history as gdh;


-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.card_click_rate as float) = (
  
  select max(cast(gdh.card_click_rate as float)) from stage.stg__gastronomades_data_history as gdh
  
);



In [0]:
%sql
-- Validação do atributo `Elementos da tela final exibidos` (end_screen_elements_shown)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.end_screen_elements_shown is null;


-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.end_screen_elements_shown as int)) as min
, max(cast(gdh.end_screen_elements_shown as int)) as max
from stage.stg__gastronomades_data_history as gdh;


-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.end_screen_elements_shown as int) = (
  
  select max(cast(gdh.end_screen_elements_shown as int)) from stage.stg__gastronomades_data_history as gdh
  
);

In [0]:
%sql
-- Validação do atributo `Cliques no elemento da tela final` (end_screen_clicks)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.end_screen_clicks is null;



-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.end_screen_clicks as int)) as min
, max(cast(gdh.end_screen_clicks as int)) as max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.end_screen_clicks as int) = (
  
  select max(cast(gdh.end_screen_clicks as int)) from stage.stg__gastronomades_data_history as gdh
  
);


In [0]:
%sql
-- Validação do atributo `Cliques por elemento da tela final mostrado (%)` (end_screen_click_rate)


-- Contagem de NULLs (se resultado = vazio, então nenhum NULL)
select
count(*)
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.end_screen_click_rate is null;



-- Mínimo e Máximo (atributo tipo int)
select
  min(cast(gdh.end_screen_click_rate as float)) as min
, max(cast(gdh.end_screen_click_rate as float)) as max
from stage.stg__gastronomades_data_history as gdh;



-- Identificação do registro com valor máximo
select 
*
from stage.stg__gastronomades_data_history as gdh
where 1=1
and cast(gdh.end_screen_click_rate as float) = (
  
  select max(cast(gdh.end_screen_click_rate as float)) from stage.stg__gastronomades_data_history as gdh
  
);

# 4. Modelagem dos dados
Nesta seção, faremos a modelagem dos dados a partir da Modelagem Dimensional, embasada na metodologia Star Schema. Dessa forma, além de apresentar as tabelas fato e dimensão que serão construídas no Data Warehouse (DW), também mostraremos o diagrama e o catálogo de dados.

## 4.1 Catálogo de dados
O catálogo de dados tem como objetivo consolidar os metadados dos dados que estão sendo utilizados neste MVP, a fim de realizar um mapeamento entre os dados brutos e os dados modelados. 


**Tabela 1:** Catálogo dos dados

| Origem | Tipo Origem | Descrição | Permite NULLs? | Destino (DW) | Tipo Destino | Intervalo |
| - | - | - | - | - | - | - |
| Conteúdo | string | Identificador único do conteúdo | N | content_id | string | NA |
| Título do vídeo | string | Título do conteúdo | N | content_title | string | NA |
| Horário de publicação do vídeo | string | Data de publicação do conteúdo | N | content_publish_date | date | 2022-11-05;	2025-03-10 |
| Duração | string | Duração do conteúdo (segundos) | N | content_duration | int | 6; 4.140 |
| Receita estimada (BRL) | string | Receita estimada (R$)| S | estimated_revenue | float | 0; 161,49 |
| Estimativa de reproduções monetizadas | string | Número de reproduções monetizadas | S | estimated_monetized_playbacks | int | 1; 7.141 |
| CPM (BRL) | string | Custo por Mil Impressões | S | cost_per_thousand_impressions | float | 0; 142,83 |
| RPM (BRL) | string | Receita por Mil Visualizações | S | revenue_per_thousand_views | float | 0; 1.758 |
| Impressões | string | Número de impressões | N | impressions | int | 0; 314.560 |
| Visualizações | string | Número de impressões | S | views | int | 0; 17.507 |
| Taxa de cliques de impressões (%) | string | Taxa de cliques de impressões (%) | S | click_through_rate | float | 0; 25 |
| Duração média da visualização | string | Duração média da visualização | S | avg_duration_view_rate | string | 00:00:00; 00:20:02  |
| Tempo de exibição (horas) | string | Tempo de exibição (horas) | S | watch_time | int | 0; 2.945,19  |
| Porcentagem visualizada média (%) | string | Taxa média de visualização (%) | S | avg_view_rate | int | 0,09; 639,86 |
| Inscritos | string | Número de inscritos obtidos | S | gained_subscribers | int | -1; 219 |
| Inscrições perdidas | string | Número de inscritos perdidos | S | lost_subscribers | int | 0; 3 |
| Compartilhamentos | string | Número de compartilhamentos | S | shares | int | 0; 115 |
| Comentários adicionados | string | Número de comentários | S | added_comments | int | 0; 8 |
| """Gostei"" (vs. ""Não gostei"") (%)" | string | Taxa de likes | S | like_rate | float | 0; 150 |
| Cards mostrados | string | Número de cards exibidos | S | shown_cards | int | 0; 50 |
| Cliques por card mostrado (%) | string | Taxa de cliques em cards (%) | S | card_click_rate | float | 0; 100 |
| Elementos da tela final exibidos | string | Número de elementos exibidos na tela final | S | end_screen_elements_shown | int | 0; 4275 |
| Cliques no elemento da tela final | string | Número de cliques em elementos exibidos na tela final | S | end_screen_clicks | int | 0; 78 |
| Cliques por elemento da tela final mostrado (%) | string | Taxa de cliques em elementos exibidos na tela final (%) | S | end_screen_click_rate | float | 0; 50 |


## 4.2 Diagrama
Os dados serão modelados a partir da Modelagem Dimensional, que é uma abordagem com foco em otimização de consultas e análises em um ambiente de Business Intelligence (BI). Nela, os dados são oranizados em tabelas Fato e Dimensão de acordo com as necessidades de negócio através da metodologia Star Schema. 

Neste contexto, serão desenvolvidas as seguintes tabelas fato e dimensão:

- Dimensão Conteúdo
- Dimensão Playlist
- Dimensão Calendário
- Fato Receita
- Fato Visualizações
- Fato Engajamento

colocar o diagrama em imagem

## 4.3 Data Warehouse (DW)
A partir dos dados brutos armazenados na camada Stage, faremos a modelagem dimensional dos dados utilizando a metodologia Star Schema, que tem como objetivo construir as tabelas fato e dimensão de acordo com as necessidades de negócio. 

### 4.3.1 Preparo do DW
Este é um passo simples que consiste em apagar todos os objetos construídos no DataWarehouse para que possamos simular em espécie de carga full das tabelas fato e dimensão.

In [0]:
#Exibe os diretórios do DW

#display(dbutils.fs.ls("dbfs:/user/hive/warehouse/datawarehouse.db/"))

In [0]:
#Remove os arquivos do DW

# tabelas dimensão
dbutils.fs.rm("dbfs:/user/hive/warehouse/datawarehouse.db/dw__content", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/datawarehouse.db/dw__calendar", recurse=True)

# tabelas fato
dbutils.fs.rm("dbfs:/user/hive/warehouse/datawarehouse.db/dw__views", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/datawarehouse.db/dw__revenue", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/datawarehouse.db/dw__engagement", recurse=True)


Out[274]: False

In [0]:
%sql
-- Exclui as tabelas para fazer a carga full

drop table if exists datawarehouse.dw__content;
drop table if exists datawarehouse.dw__calendar;
drop table if exists datawarehouse.dw__revenue;
drop table if exists datawarehouse.dw__views;

### 4.3.2 Tabelas Fato e Dimensão

#### Dimensão Conteúdo
A Dimensão Conteúdo tem como objetivo consolidar os dados de todos os conteúdos produzimos pelo canal Gastronomades, bem como seus atributos de interesse, como título, tipo, duração, data de publicação, entre outros.

In [0]:
%sql

-- Dimensão Conteúdo

-- drop
drop table if exists datawarehouse.dw__content;

-- create
create or replace table datawarehouse.dw__content as (

select distinct
  hash(gdh.content_id) as content_sk
, gdh.content_id
, gdh.content_title
, case 
    when cast(gdh.content_duration as int) <= 180 then 'short'
    when cast(gdh.content_duration as int) > 180  then 'video'
  end as content_type
, concat(lpad(floor(cast(gdh.content_duration as int) / 60), 2, '0'), ':', lpad(cast(gdh.content_duration as int) % 60, 2, '0')) as content_duration_hhmm
, case 
    when cast(gdh.content_duration as int) <= 180 then '<= 3 minutos'
    when cast(gdh.content_duration as int) between 181 and 900 then 'entre 3 e 15 minutos'
    when cast(gdh.content_duration as int) between 900 and 1500 then 'entre 15 e 25 minutos'
    when cast(gdh.content_duration as int) between 1501 and 2100 then 'entre 25 e 35 minutos'
    when cast(gdh.content_duration as int) >= 2101 then '< 35 minutos'
  end as content_duration_class
, date_format(to_date(gdh.content_publish_date, 'MMM d, yyyy'), 'yyyy-MM-dd') as content_publish_date  
from stage.stg__gastronomades_data_history as gdh
where 1=1
and gdh.content_id is not null
--and gdh.content_title = 'O QUE FAZER em Alfredo Chaves | Descubra a TERRA das CACHOEIRAS'
);

-- view
select * from datawarehouse.dw__content as con
order by 
  con.content_publish_date desc
limit 10;

content_sk,content_id,content_title,content_type,content_duration_hhmm,content_duration_class,content_publish_date
1219285056,o9AVzmOmFRg,JÁ PENSOU EM VOAR EM ALFREDO CHAVES? 🌄✨ #turismo #viagem,short,00:44,<= 3 minutos,2025-03-27
-836324732,qmOkVUeqNLo,Descubra MARECHAL FLORIANO | Um pedacinho da EUROPA no Espírito Santo,video,37:03,< 35 minutos,2025-03-23
1706400944,tOkjc0-uPA0,"Natureza, chalé e café fresco. Existe combinação melhor? 🏔️✨",short,00:10,<= 3 minutos,2025-03-17
453492613,9BEB8A9Y_zE,HOSPEDAGEM e AGROTURISMO em um só lugar,short,01:39,<= 3 minutos,2025-03-10
-990530105,IttVgyWlEls,RECIFE é o MELHOR destino gastronômico do Brasil,video,27:13,entre 25 e 35 minutos,2025-03-09
623782315,8HeDX_mFG5g,Pedacinho da EUROPA no ES 🇮🇹🇩🇪,short,01:42,<= 3 minutos,2025-03-05
-2007547265,IdS17O6T7kg,O QUE FAZER em Alfredo Chaves | Descubra a TERRA das CACHOEIRAS,video,22:31,entre 15 e 25 minutos,2025-02-16
-1620805251,t1YiLGtQozc,CABANA nas Montanhas Capixabas | Onde se HOSPEDAR em Alfredo Chaves,short,01:32,<= 3 minutos,2025-02-12
-706033252,itiyi9T_0Sk,O Que Fazer nas MONTANHAS de ALFREDO CHAVES? | Natureza e Aventuras no ES,short,01:20,<= 3 minutos,2025-02-06
-65484957,ggpWPZ9jco0,Descubra PORTO de GALINHAS | O VILAREJO mais incrível de PERNAMBUCO,video,25:49,entre 25 e 35 minutos,2025-02-02


#### Dimensão Calendário
A Dimensão Calendário tem como objetivo dimensionar atributos de data do modelo, isto é, disponibilizar outras informações associadas as datas existentes nas Tabelas Fato, como ano, semestre, mês, semana, entre outros.

In [0]:
%sql

-- Dimensão Calendário

-- drop
drop table if exists datawarehouse.dw__calendar;

-- create
create table datawarehouse.dw__calendar as (

    -- Tabela temporária: incremento para adicionar à data
    with temp__date as (
      select 
        date_add('2022-01-01', idx) as time_id
      from (
            select explode(
                            sequence(
                                      0, date_diff(current_date(),'2022-01-01'),1)) as idx
          )
    )

    -- Main
    select
      hash(cast(dat.time_id as varchar(10)))                                              as date_sk   
    , dat.time_id                                                                         as date
    , year(dat.time_id)                                                                   as year 
    , quarter(dat.time_id)                                                                as quarter
    , concat( year(dat.time_id), ', ', date_format(dat.time_id,'MMMM'))                   as year_and_month
    , concat( year(dat.time_id), ' ', substring(date_format(dat.time_id,'MMMM'),1,3))     as short_year_and_month  
    , concat( year(dat.time_id), ', ', month(dat.time_id))                                as year_and_month_number      
    , month(dat.time_id)                                                                  as month_number 
    , date_format(dat.time_id,'MMMM')                                                     as month_name
    , weekofyear(dat.time_id)                                                             as week_of_year
    , day(dat.time_id)                                                                    as day_of_month
    , concat(
              dayofweek(dat.time_id)
            , ', '
            , date_format(dat.time_id, 'EEEE')
                                )                                                         as day_of_week
    , date_format(dat.time_id, 'EEEE')                                                    as day_of_week_name                            
    , case 
        when dayofweek(dat.time_id) not in (1,7) then 'Weekend'
        else 'Weekday'
      end                                                                                 as weekday_or_weekend
    from temp__date as dat

);

-- view
select * from datawarehouse.dw__calendar
limit 3;

date_sk,date,year,quarter,year_and_month,short_year_and_month,year_and_month_number,month_number,month_name,week_of_year,day_of_month,day_of_week,day_of_week_name,weekday_or_weekend
1347365991,2022-01-01,2022,1,"2022, January",2022 Jan,"2022, 1",1,January,52,1,"7, Saturday",Saturday,Weekday
-1572948575,2022-01-02,2022,1,"2022, January",2022 Jan,"2022, 1",1,January,52,2,"1, Sunday",Sunday,Weekday
-1539929955,2022-01-03,2022,1,"2022, January",2022 Jan,"2022, 1",1,January,1,3,"2, Monday",Monday,Weekend


#### Fato Receita
A Fato Receita tem como objetivo consolidar as métricas de receita e monetização do canal Gastronomades, como estimativa de reproduções monetizadas, receita estimada, o custo que um anunciante paga para cada mil exibições de anúncios no canal e a receita gerada para o criador de conteúdo para cada mil visualizações do canal.

In [0]:
%sql

-- Fato Receita

-- drop
drop table if exists datawarehouse.dw__revenue;

-- create
create table datawarehouse.dw__revenue as (

    select
      hash(concat(gdh.reference_date, 'R', gdh.content_id))                   as revenue_id
    , hash(gdh.content_id)                                                    as content_sk
    , hash(gdh.reference_date)                                                as date_sk
    , gdh.reference_date                                                      as reference_date
    , cast(gdh.estimated_monetized_playbacks as int)                          as estimated_monetized_playbacks
    , round(cast(gdh.estimated_revenue as decimal(10,4)),4)                   as estimated_revenue
    , round(cast(gdh.cost_per_thousand_impressions  as decimal(10,4)),4)      as cpm
    , round(cast(gdh.revenue_per_thousand_views  as decimal(10,4)),4)         as rpm
    from stage.stg__gastronomades_data_history as gdh


);

-- view
select * from datawarehouse.dw__revenue
limit 3;

revenue_id,content_sk,date_sk,reference_date,estimated_monetized_playbacks,estimated_revenue,cpm,rpm
-1755916171,-1638235598,-264187109,2022-11-01,null,null,null,null
782588824,1421000376,-264187109,2022-11-01,null,null,null,null
1337947354,-593308428,-608804373,2022-12-01,null,null,null,null


#### Fato Visualizações
A Fato Visualizações tem como objetivo consolidar as métricas de visualizações de conteúdo do canal Gastronomades, como número de impressões, número de visualizações, taxa de cliques, tempo total de visualização e tempo médio de visualização.

In [0]:
%sql
-- Fato Visualizações

-- drop
drop table if exists datawarehouse.dw__views;

-- create
create table datawarehouse.dw__views as (

    select
      hash(concat(gdh.reference_date, 'V', gdh.content_id))                                             as view_id
    , hash(gdh.content_id)                                                                              as content_sk
    , hash(gdh.reference_date)                                                                          as date_sk
    , gdh.reference_date                                                                                as reference_date
    , cast(gdh.impressions as int)                                                                      as impressions
    , cast(gdh.views as int)                                                                            as views
    , cast(gdh.click_through_rate as decimal(10,2))                                                     as click_through_rate_pct --%   
    , cast(gdh.watch_time as decimal(10,4))                                                             as total_watch_time
    , concat( lpad(floor(gdh.watch_time), 2, '0')
              , ':'
              , lpad(cast(round((gdh.watch_time - floor(gdh.watch_time)) * 60, 0) as int), 2, '0'))     as total_watch_time_hhmm
    , cast(gdh.avg_view_rate as decimal(10,2))                                                          as avg_view_rate_pct -- %
    , gdh.avg_duration_view_rate                                                                        as avg_duration_view_rate
    from stage.stg__gastronomades_data_history as gdh


);

-- view
select * from datawarehouse.dw__views
limit 3;

view_id,content_sk,date_sk,reference_date,impressions,views,click_through_rate_pct,total_watch_time,total_watch_time_hhmm,avg_view_rate_pct,avg_duration_view_rate
175891426,-1638235598,-264187109,2022-11-01,588,55,5.10,1.5986,01:36,21.05,0:01:44
-1699031434,1421000376,-264187109,2022-11-01,48460,3131,4.39,257.1505,25:09,20.32,0:04:55
-600190113,-593308428,-608804373,2022-12-01,9066,671,4.99,56.4100,56:25,24.37,0:05:02


#### Fato Engajamento
A Fato Engajamento tem como objetivo consolidar as métricas de engajamento do canal Gastronomades, como número inscritos ganhados e perdidos, número de compartilhamentos, número de cards exibidos, número de elementos de tela final exibidos e taxa de cliques em cards e elementos de tela final.

In [0]:
%sql
-- Fato Engajamento

-- drop
drop table if exists datawarehouse.dw__engagement;

-- create
create table datawarehouse.dw__engagement as (

    select
      hash(concat(gdh.reference_date, 'E', gdh.content_id))          as engagement_id
    , hash(gdh.content_id)                                           as content_sk
    , hash(gdh.reference_date)                                       as date_sk
    , gdh.reference_date                                             as reference_date
    , cast(gdh.gained_subscribers as int)                            as gained_subscribers
    , cast(gdh.lost_subscribers as int)                              as lost_subscribers  
    --, likes
    --, dislikes
    , cast(gdh.like_rate as decimal(10,2))                           as like_rate_pct -- %
    , cast(gdh.shares as int)                                        as shares
    , cast(gdh.added_comments as int)                                as comments 
    , cast(gdh.shown_cards as int)                                   as shown_cards
    , cast(gdh.end_screen_elements_shown as int)                     as end_screen_elements_shown
    , cast(gdh.card_click_rate as decimal(10,2))                     as shown_cards_click_rate_pct -- %
    , cast(gdh.end_screen_click_rate as decimal(10,2))               as end_screen_click_rate_pct -- %
    from stage.stg__gastronomades_data_history as gdh


);

-- view
select * from datawarehouse.dw__engagement
limit 3;

engagement_id,content_sk,date_sk,reference_date,gained_subscribers,lost_subscribers,like_rate_pct,shares,comments,shown_cards,end_screen_elements_shown,shown_cards_click_rate_pct,end_screen_click_rate_pct
-1410781505,-1638235598,-264187109,2022-11-01,2,0,100.00,0,3,0,8,null,12.50
-64725230,1421000376,-264187109,2022-11-01,31,3,98.63,20,2,0,252,null,0.40
2080148104,-593308428,-608804373,2022-12-01,4,2,100.00,6,3,0,130,null,5.38


# 5. Análise dos dados
Esta seção tem como objetivo responder parte das perguntas de interesse levantadas na Seção 2.

## 5.1 Big Numbers


In [0]:
%sql
-- Quantos inscritos foram ganhados a partir dos conteúdos?
-- Quantos inscritos foram perdidos a partir dos conteúdos?
-- Quantos compartilhamentos foram gerados a partir dos conteúdos?
-- Quantos cards foram mostrados a partir dos conteúdos?
-- Quantos elementos em tela final foram exibidos a partir dos conteúdos?

select 
  sum(eng.gained_subscribers)                             as gained_subscribers
, sum(eng.lost_subscribers)                               as lost_subscribers
, sum(eng.gained_subscribers)-sum(eng.lost_subscribers)   as subscribers_balance
, sum(eng.shares)                                         as shares
, sum(eng.shown_cards)                                    as shown_cards
, sum(eng.end_screen_elements_shown)                      as end_screen_elements_shown
, count(distinct eng.content_sk)                          as content_count
from datawarehouse.dw__engagement as eng
inner join datawarehouse.dw__content as con on (eng.content_sk = con.content_sk)
where 1=1
and con.content_type = 'video';

## 5.2 Videos


### ENGAJAMENTO

In [0]:
%sql

-- Quantos inscritos foram ganhados a partir dos conteúdos de longa duração nos últimos 12 meses?
-- Quantos inscritos foram perdidos a partir dos conteúdos de longa duração nos últimos 12 meses?
-- Quantos compartilhamentos foram gerados a partir dos conteúdos de longa duração nos últimos 12 meses?
-- Quantos cards foram mostrados a partir dos conteúdos de longa duração nos últimos 12 meses?
-- Quantos elementos em tela final foram exibidos a partir dos conteúdos de longa duração nos últimos 12 meses?

select
*
from (
    select 
      cal.short_year_and_month
    , cal.year
    , cal.month_number    
    , sum(eng.gained_subscribers)           as gained_subscribers
    , sum(eng.lost_subscribers)             as lost_subscribers
    , sum(eng.shares)                       as shares
    , sum(eng.shown_cards)                  as shown_cards
    , sum(eng.end_screen_elements_shown)    as end_screen_elements_shown
    , count(distinct eng.content_sk)        as content_count
    from datawarehouse.dw__engagement     as eng
    inner join datawarehouse.dw__content  as con on (eng.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (eng.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and eng.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      cal.short_year_and_month
    , cal.year
    , cal.month_number 

) aux
order by
  aux.year
, aux.month_number 
;

short_year_and_month,year,month_number,gained_subscribers,lost_subscribers,shares,shown_cards,end_screen_elements_shown,content_count
2024 Mar,2024,3,50,0,143,0,2965,58
2024 Apr,2024,4,45,0,165,0,2968,59
2024 May,2024,5,67,2,200,0,4107,60
2024 Jun,2024,6,60,1,213,0,3671,62
2024 Jul,2024,7,73,2,302,54,5602,64
2024 Aug,2024,8,48,1,216,148,4605,66
2024 Sep,2024,9,53,0,162,104,3959,64
2024 Oct,2024,10,43,1,174,108,3041,66
2024 Nov,2024,11,48,1,169,81,3103,68
2024 Dec,2024,12,46,0,168,48,3086,68


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos de longa duração que geraram mais incritos nos últimos 12 meses


select
*
from (
    select 
      con.content_title                                                                             as content_title
    , dense_rank() over (order by (sum(eng.gained_subscribers)-sum(eng.lost_subscribers)) desc)     as subscribers_balance_rank  
    , sum(eng.gained_subscribers)-sum(eng.lost_subscribers)                                         as subscribers_balance    
    , sum(eng.gained_subscribers)                                                                   as gained_subscribers
    , sum(eng.lost_subscribers)                                                                     as lost_subscribers
    from datawarehouse.dw__engagement     as eng
    inner join datawarehouse.dw__content  as con on (eng.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (eng.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and eng.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.subscribers_balance_rank <=10
and aux.subscribers_balance_rank > 0
order by
  aux.subscribers_balance_rank 
limit 15  
;

content_title,subscribers_balance_rank,subscribers_balance,gained_subscribers,lost_subscribers
O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,1,277,281,4
VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😱,2,105,108,3
Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura,3,59,60,1
O que fazer em Milagres | O MELHOR da Rota Ecológica dos MILAGRES,4,56,56,0
O que fazer em SANTA TERESA | O MELHOR do Circuito Caravaggio,5,43,43,0
O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul,6,38,38,0
Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro,7,37,38,1
A MELHOR pousada de Arraial do Cabo | Onde se hospedar no Caribe Brasileiro,8,34,34,0
O que fazer em Arraial do Cabo | Quadriciclo no CARIBE BRASILEIRO,9,24,24,0
O MELHOR Parque de Trampolim de São Paulo | Impulso Park,9,24,25,1


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos de longa duração que geraram mais compartilhamentos nos últimos 12 meses


select
*
from (
    select 
      con.content_title                                        as content_title
    , dense_rank() over (order by sum(eng.shares) desc)        as shares_rank      
    , sum(eng.shares)                                          as shares
    from datawarehouse.dw__engagement     as eng
    inner join datawarehouse.dw__content  as con on (eng.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (eng.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and eng.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.shares_rank <=10
and aux.shares_rank > 0
order by
  aux.shares_rank 
limit 15  
;

content_title,shares_rank,shares
Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura,1,434
O que fazer em SANTA TERESA | O MELHOR do Circuito Caravaggio,2,193
O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,3,167
O que fazer no Circuito Caravaggio | O MELHOR de Santa Teresa,4,151
O MELHOR Parque de Trampolim de São Paulo | Impulso Park,5,131
Praia escondida em Guarapari | Descubra o Parque Morro da PESCARIA,6,115
O que fazer em VITÓRIA ES | Roteiro na Praia da Curva da Jurema,7,111
O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul,8,102
O que fazer em Arraial do Cabo | Quadriciclo no CARIBE BRASILEIRO,9,82
FESTA do MORANGO em Pedra Azul | Agroturismo nas Montanhas Capixabas,10,64


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos de longa duração que tiveram mais cards mostrados nos últimos 12 meses


select
*
from (
    select 
      con.content_title                                             as content_title
    , dense_rank() over (order by sum(eng.shown_cards) desc)        as shown_cards_rank      
    , sum(eng.shown_cards)                                          as shown_cards
    from datawarehouse.dw__engagement     as eng
    inner join datawarehouse.dw__content  as con on (eng.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (eng.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and eng.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.shown_cards_rank <=10
and aux.shown_cards_rank > 0
order by
  aux.shown_cards_rank 
limit 15  
;

content_title,shown_cards_rank,shown_cards
O que fazer em Arraial do Cabo | Quadriciclo no CARIBE BRASILEIRO,1,204
ONDE comer em CUSCO | Melhores restaurantes na PLAZA DE ARMAS,2,131
QUECHUA HOTEL EM CUSCO NO PERU,3,107
Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro,4,92
O que fazer em Milagres | O MELHOR da Rota Ecológica dos MILAGRES,5,80
O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,6,72
VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😱,7,71
Descubra PORTO de GALINHAS | O VILAREJO mais incrível de PERNAMBUCO,8,32
MORADA DOS DEUSES EM CUSCO NO PERU,9,28
O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul,10,22


Databricks visualization. Run in Databricks to view.

### VISUALIZAÇÕES

In [0]:
%sql
-- Impressões no período (últimos 12 meses)
-- Visualizações no período (últimos 12 meses)
-- Horas totais de visualização no período (últimos 12 meses)

select
*
from (
    select 
      cal.short_year_and_month
    , cal.year
    , cal.month_number    
    , sum(vie.impressions)                                  as impressions
    , sum(vie.views)                                        as views
    , round(100*(sum(vie.views)/sum(vie.impressions)),1)    as view_rate_pct
    , sum(vie.total_watch_time)                             as total_watch_time
    , floor(sum(vie.total_watch_time)) || ':' || 
      (lpad(floor((sum(vie.total_watch_time) - 
      floor(sum(vie.total_watch_time))) * 60), 2, '0'))    as total_watch_time_hh_mm
    from datawarehouse.dw__views     as vie
    inner join datawarehouse.dw__content  as con on (vie.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (vie.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and vie.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      cal.short_year_and_month
    , cal.year
    , cal.month_number 

) aux
order by
  aux.year
, aux.month_number 
;    
    

short_year_and_month,year,month_number,impressions,views,view_rate_pct,total_watch_time,total_watch_time_hh_mm
2024 Mar,2024,3,49676,7215,14.5,430.8761,430:52
2024 Apr,2024,4,49285,7779,15.8,481.6997,481:41
2024 May,2024,5,69026,10132,14.7,630.8988,630:53
2024 Jun,2024,6,59716,9739,16.3,626.0859,626:05
2024 Jul,2024,7,80960,13144,16.2,859.3309,859:19
2024 Aug,2024,8,77135,11552,15.0,745.8802,745:52
2024 Sep,2024,9,64167,9838,15.3,623.2113,623:12
2024 Oct,2024,10,50043,7951,15.9,522.8858,522:53
2024 Nov,2024,11,44960,6631,14.7,490.4154,490:24
2024 Dec,2024,12,45587,6675,14.6,547.0816,547:04


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos de longa duração que tiveram mais impressões nos últimos 12 meses

select
*
from (
    select 
      con.content_title                                                 as content_title
    , dense_rank() over (order by sum(vie.impressions) desc)            as impressions_rank      
    , sum(vie.impressions)                                              as impressions
    , sum(vie.views)                                                    as views
    , sum(vie.total_watch_time)                                         as total_watch_time
    , floor(sum(vie.total_watch_time)) || ':' || 
      (lpad(floor((sum(vie.total_watch_time) - 
      floor(sum(vie.total_watch_time))) * 60), 2, '0'))                 as total_watch_time_hh_mm    
    from datawarehouse.dw__views     as vie
    inner join datawarehouse.dw__content  as con on (vie.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (vie.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and vie.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.impressions_rank <=10
and aux.impressions_rank > 0
order by
  aux.impressions_rank 
limit 15  
;

content_title,impressions_rank,impressions,views,total_watch_time,total_watch_time_hh_mm
O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,1,396484,23107,3947.2163,3947:12
VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😱,2,116783,19849,723.0454,723:02
Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro,3,74284,6233,1175.6223,1175:37
O MELHOR Parque de Trampolim de São Paulo | Impulso Park,4,49844,6171,221.1770,221:10
O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul,5,47868,4513,529.9802,529:58
O que fazer em SANTA TERESA | O MELHOR do Circuito Caravaggio,6,45614,6776,531.3346,531:20
O que fazer em Milagres | O MELHOR da Rota Ecológica dos MILAGRES,7,43545,4016,639.0942,639:05
Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura,8,38455,10579,836.4252,836:25
O que fazer no Circuito Caravaggio | O MELHOR de Santa Teresa,9,36730,5001,393.6591,393:39
ONDE comer em CUSCO | Melhores restaurantes na PLAZA DE ARMAS,10,36396,3485,347.9438,347:56


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos de longa duração que tiveram mais visualizações nos últimos 12 meses

select
*
from (
    select 
      con.content_title                                                 as content_title
    , dense_rank() over (order by sum(vie.views) desc)                  as views_rank      
    , sum(vie.impressions)                                              as impressions
    , sum(vie.views)                                                    as views
    , sum(vie.total_watch_time)                                         as total_watch_time
    , floor(sum(vie.total_watch_time)) || ':' || 
      (lpad(floor((sum(vie.total_watch_time) - 
      floor(sum(vie.total_watch_time))) * 60), 2, '0'))                 as total_watch_time_hh_mm    
    from datawarehouse.dw__views     as vie
    inner join datawarehouse.dw__content  as con on (vie.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (vie.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and vie.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.views_rank <=10
and aux.views_rank > 0
order by
  aux.views_rank 
limit 15  
;

content_title,views_rank,impressions,views,total_watch_time,total_watch_time_hh_mm
O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,1,396484,23107,3947.2163,3947:12
VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😱,2,116783,19849,723.0454,723:02
Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura,3,38455,10579,836.4252,836:25
O que fazer em SANTA TERESA | O MELHOR do Circuito Caravaggio,4,45614,6776,531.3346,531:20
Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro,5,74284,6233,1175.6223,1175:37
O MELHOR Parque de Trampolim de São Paulo | Impulso Park,6,49844,6171,221.1770,221:10
O que fazer no Circuito Caravaggio | O MELHOR de Santa Teresa,7,36730,5001,393.6591,393:39
O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul,8,47868,4513,529.9802,529:58
O que fazer em Arraial do Cabo | Quadriciclo no CARIBE BRASILEIRO,9,28820,4231,276.9015,276:54
O que fazer em VITÓRIA ES | Roteiro na Praia da Curva da Jurema,10,24775,4108,230.2844,230:17


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos de longa duração que tiveram mais taxa de visualização nos últimos 12 meses

select
*
from (
    select 
      con.content_title                                                                               as content_title
    , dense_rank() over (order by (round(100*(sum(vie.views)/sum(vie.impressions)),1)) desc)          as rank      
    , sum(vie.impressions)                                                                            as impressions
    , sum(vie.views)                                                                                  as views
    , round(100*(sum(vie.views)/sum(vie.impressions)),1)                                              as view_rate_pct    
    , sum(vie.total_watch_time)                                                                       as total_watch_time
    , floor(sum(vie.total_watch_time)) || ':' || 
      (lpad(floor((sum(vie.total_watch_time) - 
      floor(sum(vie.total_watch_time))) * 60), 2, '0'))                                               as total_watch_time_hh_mm    
    from datawarehouse.dw__views     as vie
    inner join datawarehouse.dw__content  as con on (vie.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (vie.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and vie.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.rank <=10
order by
  aux.rank 
limit 15  
;

content_title,rank,impressions,views,view_rate_pct,total_watch_time,total_watch_time_hh_mm
AVENIDA PAULISTA NO DOMINGO (DOMINGO DO FERVOR),1,1057,834,78.9,52.7989,52:47
MORRO DO MORENO EM VILA VELHA - ES,2,1795,965,53.8,41.2226,41:13
BISTRÔ SALDANHA EM VITÓRIA - ES,3,1617,505,31.2,21.6251,21:37
O que fazer na Praia de CAMBURI | O ponto turístico mais FAMOSO de VIX,4,4324,1318,30.5,92.6826,92:40
CALÇADÃO URBANÓIDE NA RUA AUGUSTA - SÃO PAULO,5,3268,906,27.7,29.8341,29:50
Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura,6,38455,10579,27.5,836.4252,836:25
HOSPEDAGEM EM CONTAINER - CHALÉ CONTAINER SANTA TERESA - ES,7,1027,277,27.0,8.7218,8:43
ATHENAS CAFÉ E RESTAURANTE - RUA AUGUSTA - SÃO PAULO,8,2669,669,25.1,22.8258,22:49
LAVANDÁRIO ROTA DA PEDRA EM PEDRA AZUL - ES,9,4644,1128,24.3,50.5483,50:32
BRILHO DE NATAL DE DOMINGOS MARTINS NO ESPIRITO SANTO,10,4107,989,24.1,28.3998,28:23


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos de longa duração que tiveram mais horas de visualização nos últimos 12 meses

select
*
from (
    select 
      con.content_title                                                 as content_title
    , dense_rank() over (order by sum(vie.total_watch_time) desc)       as total_watch_time_rank      
    , sum(vie.impressions)                                              as impressions
    , sum(vie.views)                                                    as views
    , sum(vie.total_watch_time)                                         as total_watch_time
    , floor(sum(vie.total_watch_time)) || ':' || 
      (lpad(floor((sum(vie.total_watch_time) - 
      floor(sum(vie.total_watch_time))) * 60), 2, '0'))                 as total_watch_time_hh_mm    
    from datawarehouse.dw__views     as vie
    inner join datawarehouse.dw__content  as con on (vie.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (vie.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and vie.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.total_watch_time_rank <=10
and aux.total_watch_time_rank > 0
order by
  aux.total_watch_time_rank 
limit 15  
;

content_title,total_watch_time_rank,impressions,views,total_watch_time,total_watch_time_hh_mm
O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,1,396484,23107,3947.2163,3947:12
Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro,2,74284,6233,1175.6223,1175:37
Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura,3,38455,10579,836.4252,836:25
VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😱,4,116783,19849,723.0454,723:02
O que fazer em Milagres | O MELHOR da Rota Ecológica dos MILAGRES,5,43545,4016,639.0942,639:05
O que fazer em SANTA TERESA | O MELHOR do Circuito Caravaggio,6,45614,6776,531.3346,531:20
O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul,7,47868,4513,529.9802,529:58
O que fazer no Circuito Caravaggio | O MELHOR de Santa Teresa,8,36730,5001,393.6591,393:39
ONDE comer em CUSCO | Melhores restaurantes na PLAZA DE ARMAS,9,36396,3485,347.9438,347:56
O que fazer em Arraial do Cabo | Quadriciclo no CARIBE BRASILEIRO,10,28820,4231,276.9015,276:54


Databricks visualization. Run in Databricks to view.

### RECEITA

In [0]:
%sql


select
*
from (
    select 
      cal.short_year_and_month
    , cal.year
    , cal.month_number    
    , sum(rev.estimated_monetized_playbacks)   as estimated_monetized_playbacks
    , sum(rev.estimated_revenue)               as estimated_revenue
    , avg(rev.cpm)                             as average_cpm
    , avg(rev.rpm)                             as average_rpm
    from datawarehouse.dw__revenue     as rev
    inner join datawarehouse.dw__content  as con on (rev.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (rev.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and rev.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      cal.short_year_and_month
    , cal.year
    , cal.month_number 

) aux
order by
  aux.year
, aux.month_number 
;

short_year_and_month,year,month_number,estimated_monetized_playbacks,estimated_revenue,average_cpm,average_rpm
2024 Mar,2024,3,6230,10.9140,2.39146296,1.63643636
2024 Apr,2024,4,6426,9.9540,2.09167925,1.51757407
2024 May,2024,5,8912,13.7460,2.35305455,1.65226316
2024 Jun,2024,6,8886,13.5660,1.84476271,1.49054237
2024 Jul,2024,7,12183,16.0830,1.53704918,1.23431148
2024 Aug,2024,8,10159,15.6000,2.00896825,1.62647619
2024 Sep,2024,9,8844,14.7640,1.99983607,1.64240984
2024 Oct,2024,10,7086,11.8060,1.82276190,1.53533333
2024 Nov,2024,11,6276,13.3640,2.19808065,2.02230645
2024 Dec,2024,12,5809,10.8260,1.73507813,5.33464063


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos com maior número estimado de reproduções monetizadas nos últimos 12 meses

select
*
from (
    select 
      con.content_title                                                                           as content_title
    , dense_rank() over (order by sum(rev.estimated_monetized_playbacks) desc)                    as estimated_monetized_playbacks_rank      
    , sum(rev.estimated_monetized_playbacks)                                                      as estimated_monetized_playbacks
    , sum(rev.estimated_revenue)                                                                  as estimated_revenue
    , avg(rev.cpm)                                                                                as average_cpm
    , avg(rev.rpm)                                                                                as average_rpm 
    from datawarehouse.dw__revenue     as rev
    inner join datawarehouse.dw__content  as con on (rev.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (rev.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and rev.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.estimated_monetized_playbacks_rank <=10
order by
  aux.estimated_monetized_playbacks_rank 
limit 15  
;

content_title,estimated_monetized_playbacks_rank,estimated_monetized_playbacks,estimated_revenue,average_cpm,average_rpm
VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😱,1,18083,16.8010,1.00646154,55.70440000
Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura,2,10486,18.0110,2.00257143,1.78000000
O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,3,10313,37.8540,1.43728571,1.94942857
O que fazer em SANTA TERESA | O MELHOR do Circuito Caravaggio,4,6288,9.9770,1.78821429,1.50307143
O MELHOR Parque de Trampolim de São Paulo | Impulso Park,5,5806,8.2310,1.78642857,1.25071429
Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro,6,4541,13.2110,1.66900000,2.18475000
O que fazer no Circuito Caravaggio | O MELHOR de Santa Teresa,7,4414,6.4630,1.59721429,1.27035714
O que fazer em VITÓRIA ES | Roteiro na Praia da Curva da Jurema,8,4299,6.7540,1.85992857,1.64657143
O que fazer em Arraial do Cabo | Quadriciclo no CARIBE BRASILEIRO,9,3934,9.1090,2.17700000,2.09892857
O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul,10,3660,8.0510,1.77571429,1.64021429


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos com maior receita estimada nos últimos 12 meses

select
*
from (
    select 
      con.content_title                                                                           as content_title
    , dense_rank() over (order by sum(rev.estimated_revenue) desc)                                as estimated_revenue_rank      
    , sum(rev.estimated_monetized_playbacks)                                                      as estimated_monetized_playbacks
    , sum(rev.estimated_revenue)                                                                  as estimated_revenue
    , avg(rev.cpm)                                                                                as average_cpm
    , avg(rev.rpm)                                                                                as average_rpm 
    from datawarehouse.dw__revenue     as rev
    inner join datawarehouse.dw__content  as con on (rev.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (rev.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and rev.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.estimated_revenue_rank <=10
order by
  aux.estimated_revenue_rank 
limit 15  
;

content_title,estimated_revenue_rank,estimated_monetized_playbacks,estimated_revenue,average_cpm,average_rpm
O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,1,10313,37.8540,1.43728571,1.94942857
Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura,2,10486,18.0110,2.00257143,1.78000000
VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😱,3,18083,16.8010,1.00646154,55.70440000
Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro,4,4541,13.2110,1.66900000,2.18475000
O que fazer em SANTA TERESA | O MELHOR do Circuito Caravaggio,5,6288,9.9770,1.78821429,1.50307143
O que fazer em Milagres | O MELHOR da Rota Ecológica dos MILAGRES,6,2966,9.1930,1.60700000,2.00580000
O que fazer em Arraial do Cabo | Quadriciclo no CARIBE BRASILEIRO,7,3934,9.1090,2.17700000,2.09892857
O MELHOR Parque de Trampolim de São Paulo | Impulso Park,8,5806,8.2310,1.78642857,1.25071429
O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul,9,3660,8.0510,1.77571429,1.64021429
ONDE comer em CUSCO | Melhores restaurantes na PLAZA DE ARMAS,10,2787,7.5570,2.13072727,2.04490909


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos com maior RPM médio nos últimos 12 meses

select
*
from (
    select 
      con.content_title                                                                           as content_title
    , dense_rank() over (order by avg(rev.rpm)  desc)                                             as rank      
    , sum(rev.estimated_monetized_playbacks)                                                      as estimated_monetized_playbacks
    , sum(rev.estimated_revenue)                                                                  as estimated_revenue
    , avg(rev.cpm)                                                                                as average_cpm
    , avg(rev.rpm)                                                                                as average_rpm 
    from datawarehouse.dw__revenue     as rev
    inner join datawarehouse.dw__content  as con on (rev.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (rev.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and rev.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.rank <=10
order by
  aux.rank 
limit 15  
;

content_title,rank,estimated_monetized_playbacks,estimated_revenue,average_cpm,average_rpm
VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😱,1,18083,16.8010,1.00646154,55.70440000
Descubra MARECHAL FLORIANO | Um pedacinho da EUROPA no Espírito Santo,2,415,2.1140,1.96750000,3.10300000
PRAIA DO FORNO EM ARRAIAL DO CABO - RJ,3,53,0.1740,5.35733333,3.03107692
Como são os FAST FOODS peruanos | Mc Donald's e KFC em Cusco Peru,4,119,0.4750,3.34321429,2.87742857
FINAL DE SEMANA nas Montanhas Capixabas | Roteiro de 2 dias em Pedra Azul,5,566,1.0830,3.50018182,2.85727273
Onde se HOSPEDAR na ORLA BARDOT | Roteiros de Charme em Búzios,6,838,3.3610,3.81028571,2.78550000
GRECO HOTEL EM BÚZIOS - RJ,7,638,2.1950,3.64785714,2.72892308
CONDOMÍNIO VISTA AZUL + HOSPEDAGEM EM PEDRA AZUL - ES,8,1046,2.8060,2.55964286,2.47035714
"COMIDA DE RUA, SOUVENIR E SUPERMERCADOS EM CUSCO NO PERU",9,1714,4.9530,2.39663636,2.46818182
Não viaje para o PERU sem saber disso!,10,200,0.5210,3.55000000,2.34653846


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Top videos com maior CPM médio nos últimos 12 meses

select
*
from (
    select 
      con.content_title                                                                           as content_title
    , dense_rank() over (order by avg(rev.cpm)  desc)                                             as rank      
    , sum(rev.estimated_monetized_playbacks)                                                      as estimated_monetized_playbacks
    , sum(rev.estimated_revenue)                                                                  as estimated_revenue
    , avg(rev.cpm)                                                                                as average_cpm
    , avg(rev.rpm)                                                                                as average_rpm 
    from datawarehouse.dw__revenue     as rev
    inner join datawarehouse.dw__content  as con on (rev.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (rev.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and rev.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 12 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.rank <=10
order by
  aux.rank 
limit 15  
;

content_title,rank,estimated_monetized_playbacks,estimated_revenue,average_cpm,average_rpm
PRAIA DO FORNO EM ARRAIAL DO CABO - RJ,1,53,0.1740,5.35733333,3.03107692
Onde se HOSPEDAR na ORLA BARDOT | Roteiros de Charme em Búzios,2,838,3.3610,3.81028571,2.78550000
GRECO HOTEL EM BÚZIOS - RJ,3,638,2.1950,3.64785714,2.72892308
Não viaje para o PERU sem saber disso!,4,200,0.5210,3.55000000,2.34653846
FINAL DE SEMANA nas Montanhas Capixabas | Roteiro de 2 dias em Pedra Azul,5,566,1.0830,3.50018182,2.85727273
Como são os FAST FOODS peruanos | Mc Donald's e KFC em Cusco Peru,6,119,0.4750,3.34321429,2.87742857
QUECHUA HOTEL EM CUSCO NO PERU,7,331,0.9950,2.76744444,2.07100000
HOSPEDAGEM EM CONTAINER - CHALÉ CONTAINER SANTA TERESA - ES,8,237,0.3840,2.69416667,1.37750000
MORADA DOS DEUSES EM CUSCO NO PERU,9,163,0.3730,2.67244444,2.12040000
POUSADA SENTINELAS DO MAR EM ARRAIAL DO CABO - RJ,10,940,2.6980,2.66007143,2.28492857


Databricks visualization. Run in Databricks to view.

## 5.2 Shorts

In [0]:
%sql
-- Quantos inscritos foram ganhados a partir dos conteúdos de curta duração nos últimos 12 meses?
-- Quantos inscritos foram perdidos a partir dos conteúdos de curta duração nos últimos 12 meses?
-- Quantos compartilhamentos foram gerados a partir dos curta de longa duração nos últimos 12 meses?
-- Quantos cards foram mostrados a partir dos conteúdos de curta duração nos últimos 12 meses?
-- Quantos elementos em tela final foram exibidos a partir dos conteúdos de curta duração nos últimos 12 meses?
select
*
from (
    select 
      cal.short_year_and_month
    , cal.year
    , cal.month_number    
    , sum(eng.gained_subscribers)           as gained_subscribers
    , sum(eng.lost_subscribers)             as lost_subscribers
    , sum(eng.shares)                       as shares
    , sum(eng.shown_cards)                  as shown_cards
    , sum(eng.end_screen_elements_shown)    as end_screen_elements_shown
    , count(distinct eng.content_sk)        as content_count
    from datawarehouse.dw__engagement     as eng
    inner join datawarehouse.dw__content  as con on (eng.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (eng.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'short'
    and eng.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 3 meses fechados
    group by
      cal.short_year_and_month
    , cal.year
    , cal.month_number 

) aux
order by
  aux.year
, aux.month_number 
;

In [0]:
%sql
-- Top videos de longa duração que geraram mais incritos nos últimos 12 meses


select
*
from (
    select 
      con.content_title                                                   as content_title
    , dense_rank() over (order by sum(eng.gained_subscribers) desc)       as gained_subscribers_rank      
    , sum(eng.gained_subscribers)                                         as gained_subscribers
    , sum(eng.lost_subscribers)                                           as lost_subscribers
    , sum(eng.shares)                                                     as shares
    , sum(eng.shown_cards)                                                as shown_cards
    , sum(eng.end_screen_elements_shown)                                  as end_screen_elements_shown
    , count(distinct eng.content_sk)                                      as content_count
    from datawarehouse.dw__engagement     as eng
    inner join datawarehouse.dw__content  as con on (eng.content_sk = con.content_sk)
    inner join datawarehouse.dw__calendar as cal on (eng.date_sk = cal.date_sk)
    where 1=1
    and con.content_type = 'short'
    and eng.reference_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 3 meses fechados
    group by
      con.content_title

) aux
where 1=1
and aux.gained_subscribers_rank <=10
and aux.gained_subscribers > 0
order by
  aux.gained_subscribers_rank 
limit 15  
;

## 5.2 Bidimensional


In [0]:
%sql

-- Quantos vídeos por tipo o canal Gastronomades possui atualmente?

select 
  con.content_type
, count(*) as content_count
from datawarehouse.dw__content as con
group by
  con.content_type
order by
  con.content_type;



In [0]:
%sql

-- Quantos vídeos de longa duração foram publicados nos últimos 3 meses fechados + mês atual?


select 
  aux.short_year_and_month
, aux.content_count
from (

    select 
      cal.short_year_and_month
    , cal.year
    , cal.month_number    
    , count(*) as content_count
    from datawarehouse.dw__content as con
    left join datawarehouse.dw__calendar as cal on (hash(con.content_publish_date) = cal.date_sk)
    where 1=1
    and con.content_type = 'video'
    and con.content_publish_date >= add_months(trunc(current_date(), 'month'), -13) -- últimos 3 meses fechados
    --and con.content_publish_date >= date_sub(current_date(), 90) -- últimos 90 dias 
    group by
      cal.short_year_and_month
    , cal.year
    , cal.month_number    
    order by
      cal.year
    , cal.month_number   

) aux
order by 
  aux.year
, aux.month_number    
;

In [0]:
%sql

-- Quantos vídeos por tipo foram publicados anualmente no canal?

select 
  cal.year
, con.content_type
, count(*) as content_count
from datawarehouse.dw__content as con
left join datawarehouse.dw__calendar as cal on (hash(con.content_publish_date) = cal.date_sk)
group by
  cal.year
, con.content_type
order by
  cal.year
, con.content_type;


##### 

# 6. Considerações finais

Como mencionado inicialmente, o propósito deste MVP era realizar uma análise histórica dos dados do **Gastronomades**, um dos meus canais do YouTube, a fim de validar a efetividade de um conjunto de ações de melhoria que foram tomadas nos últimos meses, incluindo a exclusão do vídeo "VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😲" ao final do mês de setembro de 2025.

A análise histórica foi realizada segmentando os indicadores e métricas em três partes: 

- Engajamento
- Alcance e visualização
- Monetização e receita

Constatou-se um crescimento espressivo em ambas as métricas no primeiro trimestre de 2025 quando comparado com os dados de 2024, ainda que eles apresentem flutuações naturais. Entendemos que tais flutuações são normais e esperadas para o contexto, uma vez que a procura por determinados assuntos no YouTube está diretamente associada a questão temporal, ou seja, alguns temas são mais procurados que outros em determinadas épocas do ano. 

Quando analisamos o comportamento individual dos vídeos de longa duração nos últimos 12 meses fechados no que diz respeito a métricas de **Engajamento**, destacam-se:

- O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba (+)
- VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😲
- Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura (+)
- O que fazer em SANTA TERESA | O MELHOR do Circuito Caravaggio
- O que fazer em Arraial do Cabo | Quadriciclo no CARIBE BRASILEIRO
- ONDE comer em Cusco | Melhores restaurantes na PLAZA DE ARMAS
- QUECCHUA HOTEL EM CUSCO NO PERU

Ao olhar para as métricas de **Alcance e Visualização**, destacam-se:

- O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba (++)
- VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😲 (++)
- Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro (+)
- Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura 

Por fim, ao olhar para as métricas de **Monetização e Receita**, destacam-se:

- VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😲 (++)
- Conheça as MONTANHAS de GUARAPARI | O MELHOR da Rota da Ferradura (+)
- O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba (+)
- Descubra MARECHAL FLORIANO | Um pedacinho da Europa no Espírito Santo
- PRAIA DO FORNO EM ARRAIAL DO CABO - RJ


Esses foram os vídeos que apareceram no TOP 3 de cada uma das métricas, com destaque para os vídeos com (+), que indicam que o vídeo apareceu duas vezes e (++), que indica que o vídeo apareceu no topo por três vezes. Se formos um pouco menos extremistas e analisarmos o TOP 5 vídeos ao invés do TOP 3, outros vídeos se destacam:

- O que fazer em Milagres | O MELHOR da Rota Ecológica dos MILAGRES
- O que fazer no Circuito Caravaggio | O MELHOR de Santa Teresa
- O MELHOR de Parque de Trampolim de São Paulo | Impulso Park
- O que fazer nas MONTANHAS CAPIXABAS | Roteiro de 2 dias em Pedra Azul
- Como são os FAST FOODS peruanos | Mc Donald's e KFC em Cusco Peru


Em resumo, este MVP nos mostrou que mesmo após quase 7 meses desde a retirada do vídeo "VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😲", o canal ainda evidencia a presença do mesmo para as estatísticas, principalmente quando olhamos para Alcance e Monetização. No entanto, esta análise também nos mostrou que 4 dos 9 vídeos postados no canal a partir de Outubro/2024 também se destacaram positivamente nas estatísticas, que são: 

- O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba,
- Roteiro de 3 dias em MARAGOGI no ALAGOAS | Caribe Brasileiro,
- O que fazer em Milagres | O MELHOR da Rota Ecológica dos MILAGRES,
- Descubra MARECHAL FLORIANO | Um pedacinho da Europa no Espírito Santo,

com destaque para "O melhor da GASTRONOMIA CURITIBANA | Onde COMER e BEBER em Curitiba" que foi o vídeo que mais se destacou em praticamente todos os indicadores. Esta análise evidenciou que as ações de melhoria que estão sendo tomadas nos últimos meses estão tendo impacto positivo no canal Gastronomades.

Montei um simples [dashboard](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3001044570059160/2364554997492555/8075076918395346/latest.html) para que seja possível analisar visualmente os principais indicadores contidos na análise estatística dos dados. Para acessá-lo, basta clicar no link.

## 6.1 Próximos passos

Como próximos passos, além da ideia inicial que consumir os dados diretamente da API do YouTube, fica também a sugestão de continuar analisando os dados dos próximos meses de 2025, a fim de validar se o vídeo "VISITAMOS A MENOR PRAIA DE NUDISMO DO BRASIL 😲" vai continuar caindo do TOP VIDEOS de longa duração que mais se destacam no canal. 

Outra ação interessante é analisar individualmente os vídeos mais antigos do canal que também se destacaram para que possamos inferir sobre eventuais estratégias que foram aplicadas que ainda fazem com que estes vídeos se mantenham no topo mesmo depois de tanto tempo e com tão pouca produção quando comparados com os vídeos mais novos do canal.